In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv(r"C:\Users\R\Desktop\MASAI\PROJECT\P2_Car price prediction model\web-scraping-spinny.ipynb")
df.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2


In [ ]:
df.shape

# DROPING IRRELEVANT COLUMNS | ENCODING | SCALING

In [ ]:
df.drop(['brand','manufacturer'], axis = 1, inplace = True)

In [ ]:
df.head(2)

In [ ]:
cat = []
num = []

for i in df.columns:
    if df[i].dtypes == "O":
        cat.append(i)
    else:
        num.append(i)

In [ ]:
cat

In [ ]:
df = pd.get_dummies(data = df, columns= cat)

In [ ]:
df = df[['model (year)','km driven (in thousands)','mileage (in kmpl)','seating capacity (units)', 'fuel_type_Petrol','fuel_type_Diesel','fuel_type_Petrol+cng', 'transmission_Automatic', 'transmission_Manual','ground clearance (in mm)', 'boot space (in litres)','fuel tank capacity (in litres)', 'max power (in bhp)', 'price (in ₹)']]

In [ ]:
df.head()

# OUTLIER ANALYSIS

In [ ]:
for i in num:
    sns.boxplot(df[num], y = i)
    plt.show()

We can remove outliers from the above features except the seating capacity column (which is more like a categorical feature)

In [ ]:
num

In [ ]:
out = ['model (year)','km driven (in thousands)','mileage (in kmpl)','ground clearance (in mm)','boot space (in litres)',
       'fuel tank capacity (in litres)','max power (in bhp)','price (in ₹)']

for i in out:
    q1 = df[i].quantile(0.25)
    q3 = df[i].quantile(0.75)
    iqr = q3 - q1
    ul = q3 + 1.5*iqr
    ll = q1 - 1.5*iqr
    df = df[(df[i]<=ul) & (df[i]>=ll)]
    sns.boxplot(df[out], y = i)
    plt.show()

In [ ]:
df.shape

### SCALING

In [ ]:
# num

In [ ]:
# num.remove('price (in ₹)')

In [ ]:
# sc = StandardScaler()

In [ ]:
# df[num] = sc.fit_transform(df[num])

In [ ]:
# df.head(2)

### ANALYSING CORRELATION BETWEEN DIFFERENT FEATURES

In [ ]:
plt.figure(figsize=(25,15))
sns.heatmap(df[num].corr(), annot=True)

In [ ]:
sns.pairplot(df[num], diag_kind='kde')

It is clearly visible with the help of these plots that no two features are highly correlated with each other. This will help us building a better model.

# MODEL PREPARATION

In [ ]:
x = df.drop('price (in ₹)', axis=1)
y = df['price (in ₹)']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

## EVALUATING THE MODEL

### Mean Squared Error

In [ ]:
mse = metrics.mean_squared_error(y_test, y_pred)
mse

### Root Mean Squared Error

In [ ]:
rmse = np.sqrt(mse)
rmse

### R2 Score or Coefficient of Determination

In [ ]:
r2_score = metrics.r2_score(y_test, y_pred)
r2_score

### Adjusted R2 Score

Adjusted R² = 1 - [(1 - R²) * (n - 1) / (n - p - 1)]

Where:

* R² is the regular R-squared score
* n refers to the number of observations/records in the training dataset (or the sample size)
* p represents the number of predictors or independent variables in the model

In [ ]:
n = x_train.shape[0]
n

In [ ]:
p = x_train.shape[1]
p

In [ ]:
adjusted_r2_score = 1 - ((1 - r2_score)*(n-1)/(n-p-1))
adjusted_r2_score

## OUR MODEL IS PERFORMING WELL! HURREY!!!

### Now, let's apply our model in a real life scenario by predicting price of a used car based on desired features to be provided as input to the model

In [ ]:
df.head(1)

## Predicting price of a used car having:
* model (year) = 2016
* km driven (in thousands) = not more than 60k
* mileage (in kmpl) = at least 18 kmpl
* seating capacity (units) = 5
* fuel_type_ Petrol = yes
* fuel_type_ Diesel = no
* fuel_type_ Petrol+cng = no
* transmission_ Automatic = no
* transmission_ Manual = yes
* ground clearance (in mm) = not less than 180 mm
* boot space (in litres) = at least 350 litres
* fuel tank capacity (in litres) = 35 litres is enough
* max power (in bhp) = 120 bhp

In [ ]:
lr.predict([[2016,60,18,5,1,0,0,0,1,190,350,35,120]])

A used car having the above desired features and details will cost a buyer around ₹7.3 lakhs.

### This model can also be used by sellers (1st/2nd/3rd owners) to get an idea of prices they can expect

# _______________________________________THANKS___________________